## This notebook is used to: 
### i) Transform proper motions to Galactic coordinates
### ii) Correct for Galactic rotation 
### iii) Convert back to equatorial motions with respect to the surrounding medium

In [1]:
import numpy as np
import ast
from uncertainties import ufloat, umath
from uncertainties.umath import *

import astropy.coordinates as coord
import astropy.units as u

from utils import *

#### Read the information of the source from a file

In [2]:
# reading the data from the file
filename = 'BD+43_3654.txt'
try:
    with open(filename) as f:
        data = f.read()
    data = ast.literal_eval(data)
except FileNotFoundError:
    print(f"File {filename} not found.")
    data = None

print(data)

{'ra': 308.40031523381674, 'ra_err': 0.010217067, 'dec': 43.98538212854551, 'dec_err': 0.010932052, 'parallax': 0.5823258993042517, 'parallax_err': 0.012209491, 'mu_ra_cosdec': -2.594735635507606, 'mu_ra_cosdec_err': 0.013595376, 'mu_dec': 0.7286401164024895, 'mu_dec_err': 0.014304411, 'V_r': 60, 'V_r_err': 10}


#### Define a SkyCoord object using barycentric coordinates and velocity in the ICRS

In [3]:
# Gaia DR3 data
source = coord.SkyCoord(
    ra=data['ra']*u.degree, 
    dec=data['dec']*u.degree,
    distance=(data['parallax']*u.mas).to(u.kpc, u.parallax()),
    pm_ra_cosdec=data['mu_ra_cosdec']*u.mas/u.yr,
    pm_dec=data['mu_dec']*u.mas/u.yr,
    radial_velocity=data['V_r']*u.km/u.s, 
    frame='icrs'
)

#### Get the Galactic coordinates and proper motions using astropy

In [4]:
# Save in shorter-name variables. These are all adimensional.
parallax = ufloat(data['parallax'], data['parallax_err'])
ra_err = radians(data['ra_err'])
dec_err = radians(data['dec_err'])
pm_ra_err = data['mu_ra_cosdec_err']
pm_dec_err = data['mu_dec_err']
v_r_err = data['V_r_err']

# Convert to ufloatable objects (adimensional)
ra = ufloat(source.ra.radian, ra_err)
dec = ufloat(source.dec.radian, dec_err)
pm_ra = ufloat(source.pm_ra_cosdec.to('mas/yr').value, pm_ra_err)
pm_dec = ufloat(source.pm_dec.to('mas/yr').value, pm_dec_err)
V_r = ufloat(source.galactic.radial_velocity.value, v_r_err)
print(f'Galactic coordinates: RA = {ra:.5f} rad, DEC = {dec:.5f} rad')
print(f'Proper motion: mu_ra = {pm_ra:.2u} mas/yr, mu_dec = {pm_dec:.2u} mas/yr')
print(f'Radial velocity: V_r = {V_r:.2f} km/s')

# Retrieve coordinates, distance and proper motion in Galactic coordinates. These are not adimensional. 
l, b, D = source.galactic.l, source.galactic.b, source.galactic.distance
mu_l, mu_b = source.galactic.pm_l_cosb, source.galactic.pm_b

print(f'\nDistance: {D:.3f}')
print(f'Galactic coordinates: l = {l.deg:.3f}°, b = {b.deg:.3f}°')
print(f'Proper motion: mu_l = {mu_l:.2f}, mu_b = {mu_b:.2f}')

Galactic coordinates: RA = 5.38260+/-0.00018 rad, DEC = 0.76769+/-0.00019 rad
Proper motion: mu_ra = -2.595+/-0.014 mas/yr, mu_dec = 0.729+/-0.014 mas/yr
Radial velocity: V_r = 60.00+/-10.00 km/s

Distance: 1.717 kpc
Galactic coordinates: l = 82.410°, b = 2.325°
Proper motion: mu_l = -0.96 mas / yr, mu_b = 2.52 mas / yr


#### Re-do the conversion of coordinates having error propagation
(The calculations within astropy do not work with ufloat objects. Check that the results are consistent.)

In [5]:
# Define the transformation matrix between celestial to Galactic coordinates as in: 
# https://gea.esac.esa.int/archive/documentation/GDR1/Data_processing/chap_cu3ast/sec_cu3ast_intro.html

# Matrix A_G' from Eq. 3.11:
Agt = np.array([[-0.0548755604162154,-0.8734370902348850,-0.4838350155487132],\
              [0.4941094278755837,-0.4448296299600112,0.7469822444972189],\
              [-0.8676661490190047,-0.1980763734312015,0.4559837761750669]])

dist = 1./parallax

# Define the position vectors in ICRS and convert to Galactic:
cra, sra = cos(ra), sin(ra)
cd, sd = cos(dec), sin(dec)
r_icrs = np.array([cra * cd, sra * cd, sd])   # Eq. 3.7
r_gal = np.dot(Agt, r_icrs)                   # Eq. 3.9

# Galactic longitude (l) and latitude (b), from Eq. 3.13
l_gal = umath.atan2(r_gal[1], r_gal[0])   
b_gal = umath.atan2(r_gal[2], sqrt(r_gal[0]**2 + r_gal[1]**2))
cl, sl = cos(l_gal), sin(l_gal)
cb, sb = cos(b_gal), sin(b_gal)

# Auxiliary column matrices from Eqs. 3.14 and 3.15
p_icrs = np.array([-sra, cra, 0])
q_icrs = np.array([-cra * sd, -sra * sd, cd])
p_gal = np.array([-sl, cl, 0])
q_gal = np.array([-cl * sb, -sl * sb, cb])

# Transform the proper motions to Galactic coordinates
mu_icrs = p_icrs * pm_ra + q_icrs * pm_dec                     # Eq. 3.16
mu_gal = np.dot(Agt, mu_icrs)                                  # Eq. 3.18
mul, mub = np.dot(p_gal.T, mu_gal), np.dot(q_gal.T, mu_gal)  # Eq. 3.20 

print('distance =', dist, 'kpc')
print(f'Galactic longitude: l = {l_gal*180/np.pi:.1u}°')
print(f'Galactic latitude: b = {b_gal*180/np.pi:.1u}°')
print(f'mu_l = {mul:.2u} mas/yr')
print(f'mu_b = {mub:.2u} mas/yr')

distance = 1.72+/-0.04 kpc
Galactic longitude: l = 82.41+/-0.01°
Galactic latitude: b = 2.325+/-0.009°
mu_l = -0.958+/-0.014 mas/yr
mu_b = 2.519+/-0.014 mas/yr


#### Calculate local Galactic velocity field at the position of the source

In [6]:
# Define the oort constants to use (accepted: 'C07', 'B19', 'W21' )
constants = 'C07'
#constants = 'B19'
#constants = 'W21'
A, B, C, K, U, V, W = oort_constants(constants)

# Calculate Galactic rotation velocity components
V_r0, V_l0, V_b0 = galactic_velocity(dist, b_gal, l_gal, A, B, C, K, U, V, W)

# Convert velocities to proper motions
mu_l0, mu_b0 = velocity_to_propermotion(dist, V_l0, V_b0)

print('Galactic rotation in mas/yr at the location of the source:')
print(f'(\u03BC_l)0 = {mu_l0:.1u}, (\u03BC_b)0 = {mu_b0:.1u}')

Galactic rotation in mas/yr at the location of the source:
(μ_l)0 = -4.55+/-0.01, (μ_b)0 = -0.81+/-0.02


#### Calculate the object proper motion w.r.t. the local medium

In [7]:
# Proper motions w.r.t. the surrounding medium:
mu_l_corr = mul - mu_l0
mu_b_corr = mub - mu_b0

print('\nCorrected proper motion of the source w.r.t. its environment:')
print(f'\u03BC_l = {mu_l_corr:.1u}, \u03BC_b = {mu_b_corr:.1u}')

#Calculate mu_ra, mu_dec, and V_tan. We use the transpose of the transformation matrix. 
Ag = Agt.T

# Transform the proper motions to Galactic coordinates
mu_gal_corr = p_gal * mu_l_corr + q_gal * mu_b_corr                                       # Eq. 3.17
mu_icrs_corr = np.dot(Ag, mu_gal_corr)                                                    # Eq. 3.19
mu_ra_corr, mu_dec_corr = np.dot(p_icrs.T, mu_icrs_corr), np.dot(q_icrs.T, mu_icrs_corr)  # Eq. 3.21 

print(f'\nObserved proper motion: \u03BC_ra = {pm_ra:.2u} mas/yr, \u03BC_dec = {pm_dec:.2u} mas/yr')
print(f'Corrected proper motion: \u03BC_ra = {mu_ra_corr:.2u} mas/yr, \u03BC_dec = {mu_dec_corr:.2u} mas/yr')

# Calculate proper motion angle before and after correction for Galactic rotation 
angle = degrees( atan(pm_dec/pm_ra) )
angle_corr = degrees( atan(mu_dec_corr/mu_ra_corr) )

print(f'\nUncorrected angle [deg] = {angle:.2u}')
print(f'Corrected angle [deg] = {angle_corr:.2u}')

# Calculate the corrected tangential velocity
V_t = 4.74 * dist * sqrt(mu_l_corr**2 + mu_b_corr**2)
print(f'\nV_t [km/s] = {V_t:.2u}')

# Calculate corrected radial velocity
V_r_corr = V_r - V_r0
print(f'V_r [km/s] = {V_r_corr:.2u}')


Corrected proper motion of the source w.r.t. its environment:
μ_l = 3.59+/-0.02, μ_b = 3.33+/-0.02

Observed proper motion: μ_ra = -2.595+/-0.014 mas/yr, μ_dec = 0.729+/-0.014 mas/yr
Corrected proper motion: μ_ra = -0.539+/-0.025 mas/yr, μ_dec = 4.872+/-0.014 mas/yr

Uncorrected angle [deg] = -15.69+/-0.30
Corrected angle [deg] = -83.69+/-0.29

V_t [km/s] = 39.90+/-0.83
V_r [km/s] = 69+/-10


### Add a consistency check of the conversion to mu_ra_corr, mu_dec_corr using astropy (no errorbars)

In [8]:
# Proper motion in Galactic coordinates (with uncertainties)
mu_l = mu_l_corr.nominal_value * u.mas/u.yr
mu_b = mu_b_corr.nominal_value * u.mas/u.yr

# Create a SkyCoord object with the Galactic proper motion
galactic_coord = coord.SkyCoord(
    l=l, 
    b=b, 
    distance=(data['parallax']*u.mas).to(u.kpc, u.parallax()),
    pm_l_cosb=mu_l, 
    pm_b=mu_b, 
    radial_velocity=data['V_r']*u.km/u.s, 
    frame='galactic'
)

# Convert to ICRS (equatorial coordinates)
icrs_coord = galactic_coord.transform_to('icrs')

# Extract the proper motion components in ICRS
mu_ra_cosdec = icrs_coord.pm_ra_cosdec
mu_dec = icrs_coord.pm_dec

print(f'Corrected proper motion: \u03BC_ra={mu_ra_cosdec:.2f}, \u03BC_dec={mu_dec:.2f}')

Corrected proper motion: μ_ra=-0.54 mas / yr, μ_dec=4.87 mas / yr
